In [1]:
#export
from exp.nb_01 import *
from torch import FloatTensor
def get_data():
    x_train, y_train,x_test, y_test = get_minst()
    return map(FloatTensor, (x_train,y_train,x_test,y_test))

def normalize(x, m, s): return (x-m)/s

In [2]:
x_train,y_train,x_valid,y_valid = get_data()

In [3]:
print(x_train.shape)
print(x_valid.shape)

torch.Size([60000, 784])
torch.Size([10000, 784])


In [4]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(33.3184), tensor(78.5675))

In [5]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(1.3370e-08), tensor(1.))

In [7]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [9]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [10]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c


(60000, 784, tensor(10.))

## Basic Version 

In [11]:
nh = 50

In [54]:
torch.randn

NameError: name 'doc' is not defined

In [72]:
# standard xavier init
w1 = torch.randn(m,nh)
# w1 = torch.randn(m,nh)/math.sqrt(m)

b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
# w2 = torch.randn(nh,1)/math.sqrt(nh)

b2 = torch.zeros(1) 

In [73]:
w1.mean(), w1.std()

(tensor(-0.0032), tensor(0.9983))

In [74]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

AssertionError: Near zero: -0.003202695632353425

In [75]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(0.0060), tensor(1.0077))

In [80]:
w1.grad is None 

True

In [81]:
def lin(x, w, b): return x@w + b

In [82]:
t = lin(x_valid, w1, b1)

In [84]:
x_valid.shape

torch.Size([10000, 784])

In [85]:
w1.shape

torch.Size([784, 50])

In [86]:
# Walk thorugh mat mul via broadcasting steps. 
t.shape

torch.Size([10000, 50])

In [87]:
t.mean(),t.std()

(tensor(0.9264), tensor(27.2463))

In [88]:
 def relu(x): return x.clamp_min(0.)

In [89]:
relu(tensor([1,1,1,0.5,-0.4,-0.9]))

tensor([1.0000, 1.0000, 1.0000, 0.5000, 0.0000, 0.0000])

In [90]:
t = relu(lin(x_valid, w1, b1))

In [91]:
t.mean(),t.std()

(tensor(11.2503), tensor(15.9970))



From pytorch docs: a: the negative slope of the rectifier used after this layer (0 for ReLU by default)
$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from He et al: Delving Deep into Rectifiers, which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)


In [51]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [24]:
w1.mean(),w1.std()

(tensor(-7.9954e-05), tensor(0.0503))

In [25]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.5991), tensor(0.8811))

In [26]:
#export
from torch.nn import init


In [27]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [28]:
w1.mean(),w1.std()

(tensor(0.0003), tensor(0.0505))

In [29]:
def relu(x): return x.clamp_min(0.) - 0.5

In [30]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.1940), tensor(0.9283))

In [31]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [32]:
%timeit -n 10 _=model(x_valid)

40.6 ms ± 4.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


 
## Loss function: MSE


In [33]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()


In [34]:
y_train,y_valid = y_train.float(),y_valid.float()

In [35]:
preds = model(x_train)

In [36]:
preds.shape

torch.Size([42000, 1])

In [37]:
mse(preds, y_train)

tensor(23.3798)

## Gradients and backward pass


In [38]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]


In [39]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [40]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [41]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [42]:
x_train[:1000,:].shape, y_train[:10000].shape

(torch.Size([1000, 784]), torch.Size([10000]))

In [43]:
x,y = x_train[:15000,:],y_train[:15000]
forward_and_backward(x, y)

In [44]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone() 
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x.g.clone()

In [45]:
xt2 = x.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [46]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [47]:
loss = forward(xt2, y)

In [48]:
loss.backward()

In [49]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor Model

In [57]:
#export 
from torch import nn 

In [50]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [51]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [52]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]


In [53]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()


In [54]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [55]:
%time loss = model(x, y)

CPU times: user 126 ms, sys: 3.58 ms, total: 129 ms
Wall time: 31.9 ms


In [56]:
%time model.backward()

CPU times: user 2.95 s, sys: 1.32 s, total: 4.28 s
Wall time: 1.22 s


In [63]:
! python notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py
